In [2]:
# Standard data manipulation libraries
import pandas as pd
import numpy as np

# Webscraping imports
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [3]:
data = pd.read_csv('./data/data.csv')

In [4]:
data.head()

,ssvid,gap_hours,gap_distance_m,gap_implied_speed_knots,positions_per_day,vessel_class,flag,off_timestamp,off_msgid,off_lat,...,off_type,off_receiver_type,off_distance_from_shore_m,on_timestamp,on_msgid,on_lat,on_lon,on_type,on_receiver_type,on_distance_from_shore_m
0,412422839,39.433333,535.624620,0.007334,0.034917,fishing,CHN,2018-11-02T08:32:54Z,ab04ae1f-022d-1cf8-6704-79f5c18f03fe,29.940583,...,B,terrestrial,0.0,2018-11-03T23:59:40Z,d66f5755-1e80-50b7-9c24-97affcd0d30b,29.938277,122.273638,B,terrestrial,1000.0
1,247143160,80.800000,1260.751756,0.008425,2.947519,trawlers,ITA,2018-05-18T18:22:40Z,a08e856c-e011-5dab-8ae8-03a4bf0c7ba1,38.099993,...,A,terrestrial,0.0,2018-05-22T03:11:32Z,97e47f62-0784-5e70-bea4-b57c23dcab63,38.088952,13.540445,A,terrestrial,0.0
2,224231150,15.816667,9683.318918,0.330574,0.115448,set_gillnets,ESP,2018-03-22T11:28:05Z,b4fbb421-1944-5a5f-b3ae-aabb589bb8d3,43.406192,...,B,terrestrial,3000.0,2018-03-23T03:17:34Z,7137847e-305e-5a4f-8b82-ab83b181c4ab,43.354105,-8.375345,B,terrestrial,1000.0
3,413002111,14.433333,2965.614746,0.110945,0.416022,trawlers,CHN,2018-02-28T19:59:57Z,06e02215-e3b7-59ac-8504-f404fc25196c,26.573672,...,A,terrestrial,42000.0,2018-03-01T10:26:32Z,499bb4b5-a209-5f79-9718-ea361000bbdc,26.564128,120.902237,A,terrestrial,44000.0
4,247074840,16.450000,495.809447,0.016275,0.538665,trawlers,ITA,2018-04-25T07:43:16Z,248c64e1-6d7b-5553-bffe-8974fc455332,44.675090,...,A,terrestrial,0.0,2018-04-26T00:10:43Z,5659fa33-6771-584a-95a5-4d35df3f544f,44.675367,12.235438,A,terrestrial,0.0


In [5]:
# convert ssvid column to list of numbers
ssvids = data['ssvid'].values.tolist()
len(ssvids)

500000

In [6]:
# Split into 20 groups of 25,000 numbers
groups = [ssvids[x:x+25000] for x in range(0, len(ssvids), 25000)]

In [7]:
len(groups)

20

## Algorithm 1

In [8]:
# imo_reg = []
# imo_notreg = []

In [13]:
# define chromedriver location
chromedriver = '/Users/jessicaertel/Downloads/chromedriver'

# define driver object
driver = webdriver.Chrome(chromedriver)

# Open browser
driver.get('https://webaccounts.imo.org')

# Select 'Public' in drop down menu
authority = Select(driver.find_element_by_name('ctl00$cpMain$ddlAuthorityType'))
authority.select_by_value('PUBLIC')

# Enter username and password
driver.find_element_by_id("ctl00_cpMain_txtUsername").send_keys("jessicarose")
driver.find_element_by_id("ctl00_cpMain_txtPassword").send_keys("Fishing0")

# Click log in button
driver.find_element_by_id("ctl00_cpMain_btnLogin").click()

# Navigate to Global Integrated Shipping Information System
driver.find_element_by_id("ctl00_cpMain_lnkGISIS").click()
driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr/td/div[3]/a/div[2]/span[1]').click()


# Create a function that searches for each MMSI number in the IMO database
# and classifies them into registered/not registered lists 

def check_registration(lst, group_num):
    for num in lst:
        driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").send_keys(num)
        driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_btnSearchShips").click()
        time.sleep(1)
        table = driver.find_element_by_xpath('//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]')
        rows = table.find_elements(By.TAG_NAME, "tr")
        if len(rows) <= 1:
            imo_notreg.append(num)
        elif len(rows) >= 2:
            imo_reg.append(num)
        driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()
        pd.Series(imo_notreg).to_csv(f'./imo_notreg{group_num}.csv')
        pd.Series(imo_reg).to_csv(f'./imo_reg{group_num}.csv')

check_registration(groups[2], 2)


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=80.0.3987.122)


In [11]:
# Check list lengths
# Registered:389 Unregistered:3031
print(f'Registered:{len(imo_reg)} Unregistered:{len(imo_notreg)}')

Registered:389 Unregistered:3031


## Algorithm 2

- Added second if statement to catch multiple rows.
- The header of the table is also being counted as a row - increased len of if statements. One search result will have len of 2.
- used code from previous work where successful on scraping one search result
- Only remaining issue is skipping the first row of multiple search results

In [ ]:
# define chromedriver location
chromedriver = '/Users/jessicaertel/Downloads/chromedriver'

# define driver object
driver = webdriver.Chrome(chromedriver)

# Open browser
driver.get('https://webaccounts.imo.org')

# Select 'Public' in drop down menu
authority = Select(driver.find_element_by_name('ctl00$cpMain$ddlAuthorityType'))
authority.select_by_value('PUBLIC')

# Enter username and password
driver.find_element_by_id("ctl00_cpMain_txtUsername").send_keys("jessicarose")
driver.find_element_by_id("ctl00_cpMain_txtPassword").send_keys("Fishing0")

# Click log in button
driver.find_element_by_id("ctl00_cpMain_btnLogin").click()

# Navigate to Global Integrated Shipping Information System
driver.find_element_by_id("ctl00_cpMain_lnkGISIS").click()
driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr/td/div[3]/a/div[2]/span[1]').click()

# Create lists to hold scraped data
name, imo_num, flag, call_sign, mmsi, vessel_type, built, tonnage, owner = [],[],[],[],[],[],[],[],[]

count = 0
length = len(imo_reg)

# For each number in ssvids
for num in imo_reg:
    count += 1
    if count % 50 == 0:
        print(f'{count} have been scraped out of {length}', end = '\r')
        
    # Enter number into MMSI field and click search
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").send_keys(num)
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_btnSearchShips").click()
    driver.implicitly_wait(5)
    
    # define the results table and rows
    table = driver.find_element_by_id('ctl00_bodyPlaceHolder_Default_gridShips')
    rows = table.find_elements(By.TAG_NAME, "tr")
    
    # if there is only one search result
    if len(rows) == 2:
        
        # Find and click the result
        driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr').click()
#         WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH,f'/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr[{row}]')))
#         driver.find_element_by_xpath(f'//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]/tbody/tr[{row}]').click()
#         driver.find_element_by_xpath('//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]/tbody/td').click()
        
        # scrape and append the data into lists
        name.append(driver.find_element_by_id("sNameCur").text)
        imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]").text)
        flag.append(driver.find_element_by_id("sFlagCur").text)
        call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]").text)
        mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]").text)
        vessel_type.append(driver.find_element_by_id("sTypeCur").text)
        built.append(driver.find_element_by_id("sBuildDateCur").text)
        tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]").text)
        owner.append(driver.find_element_by_id("sRegOwnerCur").text)
        time.sleep(1)
        
        # return to the search results page and clear search field
        driver.execute_script("window.history.go(-1)")
        
        # Clear search field    
        driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()
        
    # if there are multiple search results, loop through each result
    elif len(rows) > 2:
        for row in range(1, len(rows)+1):

            # wait until element appears, then click it
            WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH,f'/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr[{row}]')))
            driver.find_element_by_xpath(f'//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]/tbody/tr[{row}]').click()

            # scrape and append the data into lists
            name.append(driver.find_element_by_id("sNameCur").text)
            imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]").text)
            flag.append(driver.find_element_by_id("sFlagCur").text)
            call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]").text)
            mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]").text)
            vessel_type.append(driver.find_element_by_id("sTypeCur").text)
            built.append(driver.find_element_by_id("sBuildDateCur").text)
            tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]").text)
            owner.append(driver.find_element_by_id("sRegOwnerCur").text)
            time.sleep(1)

            # return to the search results page to scrape next result
            driver.execute_script("window.history.go(-1)")

    # Clear search field
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()

In [ ]:
name, imo_num, flag, call_sign, mmsi, vessel_type, built, tonnage, owner

In [ ]:
# transform lists into pandas dataframe
imo = pd.DataFrame({'name': name,
                     'imo_num': imo_num,
                     'flag': flag,
                     'call_sign': call_sign,
                     'mmsi': mmsi,
                     'vessel_type': vessel_type,
                     'built' : built,
                     'tonnage': tonnage,
                     'owner': owner})

```Python

# define chromedriver location
chromedriver = '/Users/jessicaertel/Downloads/chromedriver'

# define driver object
driver = webdriver.Chrome(chromedriver)

# Open browser
driver.get('https://webaccounts.imo.org')

# Select 'Public' in drop down menu
authority = Select(driver.find_element_by_name('ctl00$cpMain$ddlAuthorityType'))
authority.select_by_value('PUBLIC')

# Enter username and password
driver.find_element_by_id("ctl00_cpMain_txtUsername").send_keys("jessicarose")
driver.find_element_by_id("ctl00_cpMain_txtPassword").send_keys("Fishing0")

# Click log in button
driver.find_element_by_id("ctl00_cpMain_btnLogin").click()

# Navigate to Global Integrated Shipping Information System
driver.find_element_by_id("ctl00_cpMain_lnkGISIS").click()
driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr/td/div[3]/a/div[2]/span[1]').click()


# Create a function that searches for each MMSI number in the IMO database
# and classifies them into registered/not registered lists 

imo_reg = []
imo_notreg = []

def check_reg(num):
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").send_keys(num)
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_btnSearchShips").click()
    time.sleep(1)
    #table = driver.find_element_by_id('ctl00_bodyPlaceHolder_Default_gridShips')
    #table = driver.find_element_by_class_name("gridviewer_grid")
    table = driver.find_element_by_xpath('//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]')
    rows = table.find_elements(By.TAG_NAME, "tr")
    if len(rows) <= 1:
        imo_notreg.append(num)
    elif len(rows) >= 2:
        imo_reg.append(num)
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear() 
    return None

# Iterate through each list of numbers
for mmsi in ssvids:
    check_reg(mmsi)
print(len(imo_reg), len(imo_notreg))
```

In [ ]:
# save first dataframe
success = imo
success

## References
- https://stackoverflow.com/questions/60362018/macos-catalinav-10-15-3-error-chromedriver-cannot-be-opened-because-the-de
- https://selenium-python.readthedocs.io/locating-elements.html
- https://stackoverflow.com/questions/27338742/how-do-i-send-a-delete-keystroke-to-a-text-field-using-selenium-with-python
- https://stackoverflow.com/questions/12057118/iterate-through-table-in-selenium-2-webdriver-python
- https://stackoverflow.com/questions/9671224/split-a-python-list-into-other-sublists-i-e-smaller-lists